In [21]:
import pickle
import networkx as nx
import numpy as np
from webweb import Web # pip install webweb

In [27]:
# All the movie titles in correct order
movie_titles = ["Harry Potter and the Philosopher's Stone","Harry Potter and the Chamber of Secrets",
              "Harry Potter and the Prisoner of Azkaban","Harry Potter and the Goblet of Fire",
              "Harry Potter and the Order of the Phoenix","Harry Potter and the Half-Blood Prince",
              "Harry Potter and the Deathly Hallows – Part 1","Harry Potter and the Deathly Hallows – Part 2"]
movie_titles_short = [movie_titles[i].split("Harry Potter and the ", 1)[1] for i in range(len(movie_titles))]

In [32]:
new_title = []
for x in movie_titles_short:
    new_title.append("".join(x.split()))
new_title

["Philosopher'sStone",
 'ChamberofSecrets',
 'PrisonerofAzkaban',
 'GobletofFire',
 'OrderofthePhoenix',
 'Half-BloodPrince',
 'DeathlyHallows–Part1',
 'DeathlyHallows–Part2']

In [22]:
network_list = []
for movie in np.arange(1,9):
    with open(f'./data/networks_pickled/movie_{movie}.pkl', 'rb') as f:
        G = pickle.load(f)
    network_list.append(G)

In [23]:
# # Building another copy of the network, to leave out a few attributes that don't plot very well

# G_comms = nx.DiGraph()

# # Looping through other network and adding to the network
# for node in G.nodes:             
#     G_comms.add_node(node, # node name
#                 book_nr = str(G_viz.nodes[node]['book_nr']), 
#                 house = G_viz.nodes[node]['house'], 
#                 sentiment = node_sentiment[node],
#                 degree_centrality = G.nodes[node]['degree_centrality'],
#                 in_degree_centrality = G.nodes[node]['in_degree_centrality'],     
#                 out_degree_centrality = G.nodes[node]['out_degree_centrality'],     
#                 blood = G_viz.nodes[node]['blood'],
#                 gender = G_viz.nodes[node]['gender'],
#                 species = "Human" if G_viz.nodes[node]['species'] == "Human" else "Nonhuman",
#                 community = G_viz.nodes[node]['community'])

# # Adding links
# for node in G_comms.nodes:
#     for link in G_viz.nodes[node]['links']:
#         if link in G_comms: # If other entity is in the network
#             G_comms.add_edge(node, link)
            

# print("Added {} nodes. Size of network: {}".format(len(df_wiki), len(G)))  
# print("{} edges in network.".format(len(G.edges)))


In [35]:
# Declaring webweb instance of the networks

web = Web(title="Philosopher's Stone", nx_G=network_list[0])

# movie1 = network_list[0]
# movie2 = network_list[1]
# movie3 = network_list[2]
# movie4 = network_list[3]
# movie5 = network_list[4]
# movie6 = network_list[5]
# movie7 = network_list[6]
# movie8 = network_list[7]

["Philosopher'sStone",
 'ChamberofSecrets',
 'PrisonerofAzkaban',
 'GobletofFire',
 'OrderofthePhoenix',
 'Half-BloodPrince',
 'DeathlyHallows–Part1',
 'DeathlyHallows–Part2']


web.networks.ChamberofSecrets(nx_G=network_list[1])
web.networks.PrisonerofAzkaban(nx_G=network_list[2])
web.networks.GobletofFire(nx_G=network_list[3])
web.networks.OrderofthePhoenix(nx_G=network_list[4])
web.networks.Half_BloodPrince(nx_G=network_list[5])
web.networks.DeathlyHallows_PartI(nx_G=network_list[6])
web.networks.DeathlyHallows_PartII(nx_G=network_list[7])

In [36]:
# Configuration

web.display.colorBy = 'community'
web.display.sizeBy = 'degree'
web.display.gravity = .1
web.display.charge = 10
web.display.radius = 6
web.display.linkLength = 20
web.display.linkStrength = 0.1
web.display.colorPalette = 'RdYlGn'
web.display.scaleLinkOpacity = True
web.display.scaleLinkWidth = False
web.display.nameToMatch = 'Harry'

In [37]:
# Testing
web.show()

In [38]:
# Saving 
web.save('./data/networks_pickled/final_webweb.html')